In [ ]:
import pandas as pd
import numpy as np
import networkx as nx

import os
import re
import itertools
import pickle
import json
import datetime

from tqdm.notebook import tqdm
from collections import defaultdict

# Load interaction data

In [ ]:
actions = pd.read_csv("../source_data/actions_updated.csv")[["user_id", "jd_no", "browsed", "delivered", "satisfied"]]
actions["label"] = actions[["browsed", "delivered", "satisfied"]].sum(axis=1)
actions.head()

In [ ]:
# We only consider candidates with at least 5 matching vacancies in the dataset
# This not only saves time, but also improves results later on
candidate_counts = actions[actions["label"] > 0].groupby("user_id")["jd_no"].count()
candidate_list = candidate_counts[candidate_counts >= 5].index

# Load KG

In [ ]:
G = nx.DiGraph()

with open("kg.edgelist", encoding="utf-8") as f:
    for line in tqdm(f.readlines()):
        
        if "err:error" in line:
            continue
            
        s, p, o = eval(line)
        G.add_edge(s, o, edge_type=p)

In [ ]:
# Find neighbors of each node (for quicker retrieval later on) 
all_neighbors = {}

H = G.to_undirected()

for n in tqdm(G.nodes):
    all_neighbors[n] = set(H.neighbors(n))
    
all_neighbors = {k: {i for i in v if not pd.isna(i)} for k, v in all_neighbors.items()}

# Load and clean textual data

In [ ]:
df_cv = pd.read_csv("../source_data/cv_data.csv")[["user_id", "experience"]]
df_cv.head()

In [ ]:
for row in tqdm(df_cv.itertuples(), total=4500):
    
    if type(row[2]) == str:
        cv = re.sub("\n+", " \n ", row[2]).lower()
    else:
        cv = ""
    
    nx.set_node_attributes(H, {f"zne:{row[1]}": cv}, "CV")

In [ ]:
df_jd = pd.read_csv("../source_data/jd_data.csv")[["jd_no", "jd_title", "job_description"]]
df_jd.head()

In [ ]:
for row in tqdm(df_jd.itertuples(), total=265690):
    
    if type(row[2]) == str or type(row[3]) == str:
        cv = re.sub("\n+", " \n ", row[2] + row[3]).lower()
    else:
        cv = ""
    
    nx.set_node_attributes(H, {f"zne:{row[1]}": cv}, "CV")

# Perform k random walks over the KG to find candidate-vacancy sub-graphs

In [ ]:
def k_walk(G, a, b, all_neighbors, k=6, walks=100):
    sub_graph = []
            
    # Number of walks
    for _ in range(walks):
        path = ()
        
        visited = set()
        
        prev_node = a
        old = None
        
        # Length of each walk
        for _ in range(k):
            # Choose a random neighbor of our current node
            candidates = all_neighbors[prev_node] # if prev_node in all_neighbors else set(G.neighbors(prev_node))
                
            # Don't backtrack unless absolutely necessary
            if old and len(candidates) > 1:
                candidates = candidates - set([old])
            
            # If we can reach the target from the current node, 
            # do so, as long as that wouldn't create a duplicate path
            if b in candidates and (prev_node, b) not in set(sub_graph):
                next_node = b
            else:
                if candidates:
                    next_node = np.random.choice(list(candidates))
                    
                    if next_node in visited:
                        continue
                    else:
                        visited.add(next_node)                    
                else:
                    break
            
            path += (prev_node, next_node),

            # Update
            old = prev_node
            prev_node = next_node
            
            # If we found the target, store
            if next_node == b:
                sub_graph.extend(path)
                break
                
    sg = set(sub_graph)
    sg -= {(a, b)}
    sg -= {(b, a)}
                
    H = G.edge_subgraph(sg)
               
    return H

In [ ]:
def create_jsons(G, all_neighbors, k=10, walks=50):
        
    error = ""
        
    for i, candidate in tqdm(enumerate(candidate_list), total=len(candidate_list)):
        # Find all misses for the current candidate - these can either be explicit or implicit
        # Store only the first 15, as we do not need, for example, 456 matching vacancies for a single candidate
        if f"{candidate}.csv" in os.listdir("./ground_truth"):
            continue
        
        try:
            hits = actions[(actions["user_id"] == candidate) & (actions["label"] > 0)][["jd_no", "label"]][:15]

            n_hits = len(hits)

            # Select 30-n_hits misses, so we have 30 (non-)matching vacancies per candidate
            misses = actions[((actions["user_id"] == candidate) & \
                              (actions["label"] <= 0)) | \
                             ((actions["user_id"] != candidate))][["jd_no", "label"]].sample(30-n_hits)

            record = ""

            missed_graphs = defaultdict(lambda : defaultdict(dict))
            hit_graphs = defaultdict(lambda : defaultdict(dict))

            for miss in misses.itertuples():
                graph = k_walk(H, f"zne:{candidate}", f"zne:{miss[1]}", all_neighbors, k=k, walks=walks)

                if graph:
                    record += f"{candidate},{miss[1]},{miss[2] if miss[2] <= 0 else 0}\n"    
                    missed_graphs[miss[1]] = nx.node_link_data(graph)

            for hit in hits.itertuples():
                graph = k_walk(H, f"zne:{candidate}", f"zne:{miss[1]}", all_neighbors, k=k, walks=walks)

                if graph:
                    record += f"{candidate},{hit[1]},{miss[2]}\n"    
                    hit_graphs[hit[1]] = nx.node_link_data(graph)

            with open(f"misses/{candidate}.json", "w") as f1:
                json.dump(missed_graphs, f1)

            with open(f"hits/{candidate}.json", "w") as f1:
                json.dump(hit_graphs, f1)

            with open(f"ground_truth/{candidate}.csv", "w") as f1:
                f1.write(record)
        except Exception as e:
            error = str(e)
            
        with open("log.txt", "a") as f:
            if error:
                f.write(f"{datetime.datetime.now()} - {i}/{len(candidate_list)} - error: {error}\n")
                error = ""
            else:
                f.write(f"{datetime.datetime.now()} - {i}/{len(candidate_list)} - successfully written\n")

In [ ]:
create_jsons(H, all_neighbors, k=7, walks=25)